In [35]:
import math

import geojson
import requests
import tqdm

In [69]:
#GLOSSIS
dataset = 'glossis'
locations_url = 'http://pl-tc012.xtr.deltares.nl:8080/FewsWebServices/rest/digitaledelta/2.0/locations'
nodes_url = 'http://pl-tc012.xtr.deltares.nl:8080/FewsWebServices/rest/digitaledelta/2.0/nodes'
observation_types_url = 'http://pl-tc012.xtr.deltares.nl:8080/FewsWebServices/rest/digitaledelta/2.0/observationTypes'
timeseries_url = 'http://pl-tc012.xtr.deltares.nl:8080/FewsWebServices/rest/digitaledelta/2.0/timeseries'

In [70]:
# GLOFFIS
dataset = 'gloffis'
locations_url = 'http://tw-151.xtr.deltares.nl:8081/FewsWebServices/rest/digitaledelta/2.0/locations'
nodes_url = 'http://tw-151.xtr.deltares.nl:8081/FewsWebServices/rest/digitaledelta/2.0/nodes'
observation_types_url = 'http://tw-151.xtr.deltares.nl:8081/FewsWebServices/rest/digitaledelta/2.0/observationtypes'
timeseries_url = 'http://tw-151.xtr.deltares.nl:8081/FewsWebServices/rest/digitaledelta/2.0/timeseries'

In [68]:
url = locations_url
print(requests.get(locations_url))
resp = requests.get(url).json()

<Response [200]>


In [39]:
# get the first request and inspect it to get the paging information

def paging_iter(url):
    """iterate over all pages in a url"""
    print(requests.get(url))

    resp = requests.get(url).json()
    page_size = resp['paging']['maxPageSize']
    count = resp['paging']['totalObjectCount']
    # Now loop over all pages
    n = math.ceil(count / page_size)

    # store each json result
    # progress please
    for i in tqdm.tqdm_notebook(range(n)):
        # fire the request. It's not faster in parallel (tested with asyncio), so I keep it serial.
        resp = requests.get(url, dict(page=i + 1, pageSize=page_size))
        yield resp.json()
    

In [40]:
locations_resps = list(paging_iter(locations_url))

<Response [200]>


In [41]:
# collect information about the information per location
# one request per location
timeseries_resps = list(paging_iter(timeseries_url))

<Response [200]>


In [42]:
# collect all the features
features = []
for resp in locations_resps:
    results = resp['results']
    for feature in results:
        feature['id'] = feature['properties']['locationId']
        features.append(feature)        
    
features_by_id = {
    feature['id']: feature
    for feature 
    in features
}

timeseries = []
for resp in timeseries_resps:
    timeseries.extend(resp['results'])

In [72]:
all_quantities = []

for series in timeseries:
    id = series['location']['properties']['locationId']
    feature = features_by_id[id]
    quantities = feature['properties'].get('quantities', [])
    if dataset == 'glossis':
        quantity = series['observationType']['quantity']
    elif dataset == 'gloffis':
        quantity = series['observationType']['parameterCode']
    if quantity not in quantities:
        quantities.append(quantity)
    if quantity not in all_quantities:
        all_quantities.append(quantity)    
    
print(all_quantities)

['Q.obs', 'Q.simulated', 'EPot', 'T', 'P']


In [73]:
# Add Booleans for all quantities to each feature
for feature in features:
    for quantity in all_quantities:
        if quantity in feature["properties"]["quantities"]:
            feature["properties"][quantity] = True
        else:
            feature["properties"][quantity] = False
    features_by_id[feature['id']] = feature

In [67]:
# store the features
collection = geojson.FeatureCollection(features=list(features_by_id.values()))
with open('tw-151.geojson', 'w') as f:
    geojson.dump(collection, f)

In [ ]:
# Create tiles until level 12 or something.... with tippecanoe

# let's upload it
!mapbox upload siggyf.pl-tc012 pl-tc012.geojson